## **Writeups: I Dream of Genni**
Link: https://github.com/project-sekai-ctf/sekaictf-2025/tree/main/crypto/i-dream-of-genni

Xem thêm: Vampire number (https://en.wikipedia.org/wiki/Vampire_number)


#### **1. Thử thách**

Ta cần tìm kiếm hai số tự nhiên `x` có **8** chữ số và `y` có **7** chữ số sao cho với $k_i$ là chữ số thứ $i$ từ phải sang trái của `k`. Ta có:
$$
x * y = \overline{x_8(x_7*y_7)...(x_1*y_1)}
$$



#### **1.2. Ý tưởng Brute-force**
Ý tưởng Brute-force không khả thi vì với `x` có 8 chữ số, `y` có 7 chữ số, ta có `x*y` có:
$$
9.10^{7}.9.10^{6} = 81.10^{13}    \text{(case)}
$$
Do đó, không khả thi khi tính toán.

### **2. Ý tưởng `prefix-pruned backtracking`**
Kết hợp giữa kỹ thuật **cắt tỉa nhánh DFS (Pruning in DFS)** và kỹ thuật **Backtracking**.\
Thay vì thử tất cả các số, ta xây dựng dần từng chữ số, và loại bỏ sớm những hướng sai.\
Giống như ta đang đi trong mê cung, mỗi bước, ta chọn đi trái hay phải (thêm một chữ số), nhưng trước khi đi sâu, ta kiểm tra phía trước xem con đường đó có thể dẫn tới đích hay không. Nếu thấy “vô vọng”, ta quay lại ngay.

#### **2.1. Xây dựng cây duyệt sâu**
Ta cần xây dựng 2 giá trị, gồm giá trị *thực*, và giá trị *ảo*, cụ thể, xét qua ví dụ sau:
- 1. **“Thế giới thật”** – khoảng giá trị có thể có của `x * y`. Ví dụ, ban đầu ta chọn `x_pre = 123` và `y_pre = 45`, như vậy, ta có khoảng tích giới hạn cho kết quả chính xác là:
$$
123.10^5 * 45.10^5 = 5535.10^{10} \le x \times y \ge 12399999.459999 = 57,039,983.10^{6} + 1
$$

- 2. **“Thế giới mơ”** – khoảng giá trị có thể có của `dream_multiply(x, y)`. Với `x_pre`, `y_pre` ở trên, ta có:
$$
    1815.10^{10} \le \text{dream}(x, y) \ge 18158181818181
$$
Giao của hai giá trị thực và ảo là khoảng giá trị có thể của `x` và `y`. Nếu giữa 2 giá trị không có giao, ta bỏ qua, không xét theo chiều sâu tiếp tục mà quay về ban đầu. 

Như vậy, thay vì phải Brute-force, ta sử dụng cắt tỉa và Backtracking sẽ giảm trường hợp cần duyệt đi rất nhiều.

### **3. Solution code**
Nhớ rằng với `x` nhỏ nhất, `y` nhỏ nhất, ta có: 
$$
x_{min} \times y_{min} = 10^{7} \times 10^{6} = 10^{13}
x_{max} \times y_{max} = (10^{8}-1) \times (10^{7}-1) < 10^{15}
$$
Có ít nhất 14 chữ số và có nhiều nhất 16 chữ số, bỏ đi 1 chữ số của `x` ban đầu. Như vậy, với 7 cặp số $(x_i \times y_i)$ chỉ cho phép tồn tại 1 cặp số sao cho tích của chúng **chỉ có 1 chữ số**.


In [33]:
from hashlib import sha256
from Crypto.Cipher import AES

CT_HEX = '75bd1089b2248540e3406aa014dc2b5add4fb83ffdc54d09beb878bbb0d42717e9cc6114311767dd9f3b8b070b359a1ac2eb695cd31f435680ea885e85690f89'
FORBIDDEN = 381404224402842     # Exception example number

# Phương thức xử lý phép nhân thực:
def realMul(xp, xr, yp, yr):
    ''' Trong đó, 
    - `xp` là tiền tố của `x`
    - `xr` là số chữ số còn lại của x
    '''
    x_min, x_max = xp * 10**xr, (xp + 1) * 10**xr - 1
    y_min, y_max = yp * 10**yr, (yp + 1) * 10**yr - 1
    return x_min * y_min, x_max * y_max

# Phương thức xử lý phép nhân mơ ước:
def dreamMul(pre_str, remain_digits):
    min = int(pre_str) * 10**remain_digits
    max = min + 10**(remain_digits) - 1
    return min, max

# Dictionary chứa các cặp giá trị lần lượt (xi, yi) sao cho xi * yi tương ứng có 1 chữ số hoặc 2 chữ số 
PAIRS_BY_LEN = {
    1: [(a, b) for a in range(10) for b in range(10) if a * b < 10],
    2: [(a, b) for a in range(10) for b in range(10) if 10 <= a * b <= 81],
}

# Danh sách các trường hợp có thể với số chữ số mà xi * yi có thể
LEN_PATTERNS = [[2]*7] + [[2]*i + [1] + [2]*(6-i) for i in range(7)]

def dfsSearchPat(len_pattern):
    results = []
    def recursion(idx: int, dream_str: str, xp: int, yp: int):
        # print(f"Đang chạy {idx}, {dream_str}, {xp}, {yp}")
        """Đệ quy xây dựng dần hai số nguyên x và y, đồng thời kiểm tra điều kiện hợp lệ trên từng bước.

        Args:
            idx (int): Chỉ số hiện tại của chữ số đang xử lý (`xi`, `yi`), với x bỏ qua chữ số đầu tiên.
            dream_str (str): Chuỗi tạm biểu diễn kết quả "dream".
            xp (int): Phần đầu hiện tại của số x.
            yp (int): Phần đầu hiện tại của số y.

        Returns:
            
        """

        if idx == 7:                                # Đã duyệt hết
            if xp < 10_000_000 or yp < 1_000_000:   # x, y không đủ chữ số
                return 
            N = int(dream_str)
            if N != FORBIDDEN and xp * yp == N:
                results.append((xp, yp))
            return 

        # Số lượng chữ số tiếp theo được thêm:
        len_concat = len_pattern[idx]
        
        for xi, yi in PAIRS_BY_LEN[len_concat]:
            if idx == 0 and yi == 0:        # Bỏ case chữ số đầu tiên của y là 0
                continue
            xp_new = xp * 10 + xi
            yp_new = yp * 10 + yi
            dream_new = dream_str + str(xi*yi)
            tail_digits = sum(len_pattern[(idx+1):]) # Bằng số chữ số của các vị trí còn lại
            
            xr = 7 - (idx + 1)
            yr = 7 - (idx + 1)
            
            real_l, real_h = realMul(xp_new, xr, yp_new, yr)
            dream_l, dream_h = dreamMul(dream_new, tail_digits)
            
            if real_h < dream_l or real_l > dream_h:    # giao = empty
                continue
            
            quote = 10 ** tail_digits
            if not (real_l//quote <= int(dream_new) <= real_h//quote):
                continue
            
            # Ngược lại nếu thỏa mãn ==> Đệ quy:
            recursion(idx+1, dream_new, xp_new, yp_new)
    
    for x0 in range(1, 10):
        recursion(0, str(x0), x0, 0)
        
    return results


# Main:
# if __name__ == "__main__":
RES = []
for len_pattern in LEN_PATTERNS:
    RES = dfsSearchPat(len_pattern)
    print(RES)
    if len(RES) != 0:
        break

if len(RES) != 0:
    x, y = RES[0]
    key = sha256(str((x, y)).encode()).digest()
    flag = AES.new(key, AES.MODE_ECB).decrypt(bytes.fromhex(CT_HEX)).decode(errors="ignore")
    print(f"FLAG: {flag}")
    print(f"x: {x}, y:{y}")
    
else:
    print("No finding x, y!")

[(49228443, 9773647)]
FLAG: Here is your flag: SEKAI{iSOgenni_in_mY_D1234M5;_iS_it_T00_s00n}
x: 49228443, y:9773647


In [23]:
res = [[2]*7] + [[2]*i + [1] + [2]*(6-i) for i in range(7)]
print(res)

[[2, 2, 2, 2, 2, 2, 2], [1, 2, 2, 2, 2, 2, 2], [2, 1, 2, 2, 2, 2, 2], [2, 2, 1, 2, 2, 2, 2], [2, 2, 2, 1, 2, 2, 2], [2, 2, 2, 2, 1, 2, 2], [2, 2, 2, 2, 2, 1, 2], [2, 2, 2, 2, 2, 2, 1]]


In [ ]:
from hashlib import sha256
from Crypto.Cipher import AES

CT_HEX = (
    "75bd1089b2248540e3406aa014dc2b5add4fb83ffdc54d09beb878bbb0d42717"
    "e9cc6114311767dd9f3b8b070b359a1ac2eb695cd31f435680ea885e85690f89"
)

FORBIDDEN = 381404224402842  

def product_interval(xp, remx, yp, remy):
    powx = 10 ** remx
    powy = 10 ** remy
    Xmin, Xmax = xp * powx, xp * powx + (powx - 1)
    Ymin, Ymax = yp * powy, yp * powy + (powy - 1)
    return Xmin * Ymin, Xmax * Ymax

def prefix_interval(dream_prefix_str, remaining_digits):
    base = int(dream_prefix_str)
    lo = base * (10 ** remaining_digits)
    hi = (base + 1) * (10 ** remaining_digits) - 1
    return lo, hi

PAIRS_BY_LEN = {
    1: [(a, b) for a in range(10) for b in range(10) if a * b < 10],
    2: [(a, b) for a in range(10) for b in range(10) if 10 <= a * b <= 81],
}
PATTERNS = [[2] * 7] + [[2] * i + [1] + [2] * (6 - i) for i in range(7)]

def solve():
    solutions = []

    def dfs_for_pattern(pattern):
        for d0 in range(1, 10):
            def rec(i, dream_str, xp, yp):
                if i == 7:
                    if xp < 10_000_000 or yp < 1_000_000:
                        return
                    N = int(dream_str)
                    if N != FORBIDDEN and xp * yp == N:
                        solutions.append((xp, yp, N))
                    return

                need_len = pattern[i]
                for a, b in PAIRS_BY_LEN[need_len]:
                    if i == 0 and b == 0:
                        continue
                    new_xp = xp * 10 + a
                    new_yp = yp * 10 + b
                    block = str(a * b)
                    new_dream = dream_str + block

                    dream_tail_digits = sum(pattern[i + 1 :])

                    remx = 7 - (i + 1)
                    remy = 7 - (i + 1)

                    p_lo, p_hi = product_interval(new_xp, remx, new_yp, remy)
                    n_lo, n_hi = prefix_interval(new_dream, dream_tail_digits)
                    if p_hi < n_lo or p_lo > n_hi:
                        continue

                    scale = 10 ** dream_tail_digits
                    if not (p_lo // scale <= int(new_dream) <= p_hi // scale):
                        continue

                    rec(i + 1, new_dream, new_xp, new_yp)

            rec(0, str(d0), d0, 0)

    for patt in PATTERNS:
        dfs_for_pattern(patt)
        if solutions:
            break

    if not solutions:
        return None

    x, y, prod = solutions[0]
    key = sha256(str((x, y)).encode()).digest()
    flag = AES.new(key, AES.MODE_ECB).decrypt(bytes.fromhex(CT_HEX)).decode(errors="ignore")
    return (x, y, prod, flag.strip())

out = solve()
if not out:
    print("wrong")
else:
    x, y, prod, flag = out
    print(f"Found x={x}, y={y}")
    print(f"dream(x,y) = x*y = {prod}")
    print(f"Flag: {flag}")